# KHSIC approach for disentangling content and style

## Data Prep

In [1]:
import time

In [2]:
%%time

alpha = 1.0
alpha_sk = 0.5 # for creating skewed data used to learn R
eta = 0.95
batch_size = 128
ns = 1 #specify number of style features


import numpy as np
from sklearn.linear_model import LogisticRegression
import torch
from numpy import load
import sys, json
from itertools import product
from sklearn import preprocessing


# Function for binarizing labels
def binarize(y):    
    y = np.copy(y) > 5
    return y.astype(int)

# Function for creating spurious correlations on 
def create_spurious_corr(z, z_t, y_og, spu_corr= 0.1, binarize_label=True):
    y_bin = binarize(y_og)
    mod_labels = np.logical_xor(y_bin, np.random.binomial(1, spu_corr, size=len(y_bin)))
    
    modified_images = z_t[mod_labels]
    unmodified_images = z[~mod_labels]
    all_z = np.concatenate((modified_images, unmodified_images), axis=0)
    style_labels = np.concatenate((np.zeros(len(modified_images)), np.ones(len(unmodified_images))), axis=None)
    
    all_img_labels = None
    
    if binarize_label:
        modified_imgs_labels = y_bin[mod_labels]
        unmodified_imgs_labels = y_bin[~mod_labels]
        all_img_labels = np.concatenate((modified_imgs_labels, unmodified_imgs_labels), axis=None)
    else:
        modified_imgs_labels = y_og[mod_labels]
        unmodified_imgs_labels = y_og[~mod_labels]
        all_img_labels = np.concatenate((modified_imgs_labels, unmodified_imgs_labels), axis=None)    
        
    return all_z, all_img_labels, style_labels.astype(int)


# Load saved image features

z_train_og = load('./data/Z_train_og_cifar10_simclr.npy')
z_train_t = load('./data/Z_train_rotated_cifar10_simclr.npy')

z_test_og = load('./data/Z_test_og_cifar10_simclr.npy')
z_test_t = load('./data/Z_test_rotated_cifar10_simclr.npy')

y_train_og = load('./data/train_labels_cifar10.npy')

y_test_og = load('./data/test_labels_cifar10.npy')


# Create spurious correlations on train and test sets

z_train_sk, train_labels_sk, t_labels_sk = create_spurious_corr(z_train_og, z_train_t, y_train_og, 
                                         spu_corr= alpha_sk, binarize_label=False)

z_train, train_labels, _ = create_spurious_corr(z_train_og, z_train_t, y_train_og, 
                                         spu_corr= alpha, binarize_label=False)

z_test_indist, indist_test_labels, _ = create_spurious_corr(z_test_og, z_test_t, y_test_og, 
                                                         spu_corr= alpha, binarize_label=False)

z_test_ood, ood_test_labels, _ = create_spurious_corr(z_test_og, z_test_t, y_test_og, 
                                                         spu_corr= 1-alpha, binarize_label=False)



# concatenate original and colored features
z_train_og_t = np.concatenate((z_train_og, z_train_t), axis=0)
t_train_labels = np.concatenate((np.zeros(len(z_train_og)), np.ones(len(z_train_t))), axis=None) 
z_test_og_t = np.concatenate((z_test_og, z_test_t), axis=0)
t_test_labels = np.concatenate((np.zeros(len(z_test_og)), np.ones(len(z_test_t))), axis=None) 


# # concatenate features with sytle labels..style labels are in column 0
# t_labels_z_train_og_t = np.concatenate((t_train_labels.reshape(-1,1), z_train_og_t), axis=1)

# # shuffle data in t_labels_z_train_og_t
# np.random.shuffle(t_labels_z_train_og_t)

# shuffled_train_og_t = t_labels_z_train_og_t[:,1:]
# shuffled_t_train_labels = t_labels_z_train_og_t[:,:1]


/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 6.84 s, sys: 9.96 s, total: 16.8 s
Wall time: 17.1 s


## MNIST data class distribustions per domain

In [3]:
import pandas as pd

# class distribution in original and colored images - class distribution is skewed
style_and_img_labels_z_train_sk_df = pd.DataFrame(np.concatenate((t_labels_sk.reshape(-1,1),
                                                                  train_labels_sk.reshape(-1,1),z_train_sk), axis=1))

print("class distribution - column 1 is class labels - column 0 is domain/environment labels")
class_distribution_per_domain = style_and_img_labels_z_train_sk_df.groupby([1,0]).count().iloc[:,0:1]
display(class_distribution_per_domain)


# shuffle data in style_and_img_labels_z_train_sk_df
style_and_img_labels_z_train_sk = style_and_img_labels_z_train_sk_df.to_numpy()
np.random.shuffle(style_and_img_labels_z_train_sk)

shuffled_train_og_t = style_and_img_labels_z_train_sk[:,2:]
shuffled_t_train_labels = style_and_img_labels_z_train_sk[:,:1]




class distribution - column 1 is class labels - column 0 is domain/environment labels


2
1   0        
0.0 0.0  2478
    1.0  2522
1.0 0.0  2469
    1.0  2531
2.0 0.0  2455
    1.0  2545
3.0 0.0  2447
    1.0  2553
4.0 0.0  2500
    1.0  2500
5.0 0.0  2526
    1.0  2474
6.0 0.0  2500
    1.0  2500
7.0 0.0  2524
    1.0  2476
8.0 0.0  2494
    1.0  2506
9.0 0.0  2432
    1.0  2568

# Find rotation matrix R by optimization----using KHSIC loss

In [4]:
%%time

import torch
import mctorch.nn as mnn
import mctorch.optim as moptim
from hsic_calculator import HSIC, normalized_HSIC


# # Reduce the samples size
# shuffled_train_og_t = shuffled_train_og_t[:10000]
# shuffled_t_train_labels = shuffled_t_train_labels[:10000]

dtype = torch.FloatTensor
n = shuffled_train_og_t.shape[0]
d = shuffled_train_og_t.shape[1]
k = int(shuffled_train_og_t.shape[1]*eta) # % of original number of features


# Initialize R
R = mnn.Parameter(manifold=mnn.Stiefel(d,k)).float()

# print("Initial R")
# display(R)

# Define Objective function 
def obj(z, e, W, n_s=1):
    z = torch.from_numpy(z).float()
    e = torch.from_numpy(e).float()
    MI_content_style = normalized_HSIC(torch.matmul(z, W[:,:n_s]), torch.matmul(z, W[:,n_s:]))
    MI_conten_env = normalized_HSIC(torch.matmul(z,W[:,n_s:]), e)
    MI_style_env = normalized_HSIC(torch.matmul(z,W[:,:n_s]), e)
    loss = (MI_content_style + MI_conten_env) - MI_style_env
    return loss

# Optimize - passing data in mini-batches
optimizer = moptim.rAdagrad(params = [R], lr=1e-2)

best_loss = 1e5
checkpoint = {}
for epoch in range(200):
    for index in range(0, len(shuffled_train_og_t), batch_size):
        train_data_subset = shuffled_train_og_t[index:index+batch_size]
        style_labels_subset = shuffled_t_train_labels[index:index+batch_size]
        loss = obj(train_data_subset, style_labels_subset, R, ns)        
        # saving R with the smallest loss value so far
        if loss < best_loss:
            best_loss = loss
            print("Saving R, at epoch ", epoch)
            checkpoint = {'epoch': epoch, 'loss': loss, 'R': R}
            torch.save(checkpoint, 'checkpoint') 
            print("loss: ", loss)            
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

checkpoint

# print("R after optimization")
# display(R)
# (R.T)@R

/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/mctorch/nn/manifolds/stiefel.py:50: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1980.)
  q, r = torch.qr(X)


Saving R, at epoch  0
loss:  tensor(0.3852, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.2537, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.1567, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.1290, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(0.0723, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.0894, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.1064, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.3025, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.3305, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.3721, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.4194, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.4585, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.6638, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.6849, grad_fn=<SubBackward0>)
Saving R, at epoch  0
loss:  tensor(-0.

Saving R, at epoch  83
loss:  tensor(-0.9428, grad_fn=<SubBackward0>)
Saving R, at epoch  84
loss:  tensor(-0.9429, grad_fn=<SubBackward0>)
Saving R, at epoch  85
loss:  tensor(-0.9430, grad_fn=<SubBackward0>)
Saving R, at epoch  86
loss:  tensor(-0.9430, grad_fn=<SubBackward0>)
Saving R, at epoch  87
loss:  tensor(-0.9430, grad_fn=<SubBackward0>)
Saving R, at epoch  88
loss:  tensor(-0.9431, grad_fn=<SubBackward0>)
Saving R, at epoch  89
loss:  tensor(-0.9431, grad_fn=<SubBackward0>)
Saving R, at epoch  90
loss:  tensor(-0.9431, grad_fn=<SubBackward0>)
Saving R, at epoch  91
loss:  tensor(-0.9432, grad_fn=<SubBackward0>)
Saving R, at epoch  92
loss:  tensor(-0.9432, grad_fn=<SubBackward0>)
Saving R, at epoch  93
loss:  tensor(-0.9432, grad_fn=<SubBackward0>)
Saving R, at epoch  94
loss:  tensor(-0.9433, grad_fn=<SubBackward0>)
Saving R, at epoch  95
loss:  tensor(-0.9433, grad_fn=<SubBackward0>)
Saving R, at epoch  96
loss:  tensor(-0.9433, grad_fn=<SubBackward0>)
Saving R, at epoch  

{'epoch': 199,
 'loss': tensor(-0.9438, grad_fn=<SubBackward0>),
 'R': Parameter containing:
 tensor([[ 0.0585, -0.0051, -0.0013,  ..., -0.0670, -0.0335,  0.0252],
         [ 0.0199, -0.0271, -0.0200,  ...,  0.0200,  0.0286,  0.0428],
         [-0.0763,  0.0617, -0.0804,  ...,  0.0485,  0.0287, -0.0919],
         ...,
         [ 0.0047,  0.0458, -0.0240,  ..., -0.0502,  0.0284, -0.0324],
         [ 0.1188,  0.0250,  0.0311,  ..., -0.0002, -0.0710,  0.0432],
         [-0.0208,  0.0348,  0.0785,  ..., -0.0016,  0.0422, -0.0278]],
        requires_grad=True)}

## Use the obtained rotation matrix R to disentangle content and style for OOD generalization

In [5]:
%%time

# Load saved R
R_mat = torch.load('checkpoint')['R']

# Obtain post-processed features
f_train_og = z_train_og @ R_mat.detach().numpy()  
f_train = z_train @ R_mat.detach().numpy()
f_test_indist = z_test_indist @ R_mat.detach().numpy()
f_test_ood = z_test_ood @ R_mat.detach().numpy()
f_test_og = z_test_og @ R_mat.detach().numpy()
f_test_t = z_test_t @ R_mat.detach().numpy()
f_test_og_t = z_test_og_t @ R_mat.detach().numpy()



########### Baseline results ###############
# Prediction Accuracies on image features extracted using a baseline model (mlp)
logistic_regression_on_baseline = LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                              random_state=0).fit(z_train,train_labels)                                                                                     
baseline_accuracy0 = logistic_regression_on_baseline.score(z_train, train_labels)
baseline_accuracy1 = logistic_regression_on_baseline.score(z_test_indist, indist_test_labels)
baseline_accuracy2 = logistic_regression_on_baseline.score(z_test_ood, ood_test_labels)   

# Trained on original baseline features, tested on colored features - no spurious correlations here
logistic_regression_on_baseline_og = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                                                 random_state=0).fit(z_train_og,y_train_og)                                                                                     
baseline_og_accuracy0 = logistic_regression_on_baseline_og.score(z_train_og, y_train_og)
baseline_og_accuracy1 = logistic_regression_on_baseline_og.score(z_test_og, y_test_og)
baseline_transf_accuracy2 = logistic_regression_on_baseline_og.score(z_test_t, y_test_og)
####################################

# Classification task using all post-processed features except style features   
lr_model_new_HSIC_sp = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                                    random_state=0).fit(f_train[:,1:],train_labels)
new_HSIC_sp_accuracy0 = lr_model_new_HSIC_sp.score(f_train[:,1:], train_labels)
new_HSIC_sp_accuracy1 = lr_model_new_HSIC_sp.score(f_test_indist[:,1:], indist_test_labels)
new_HSIC_sp_accuracy2 = lr_model_new_HSIC_sp.score(f_test_ood[:,1:], ood_test_labels)

# trained on original post-processed features, tested on transformed post-processed 
# features without style features   
lr_model_new_HSIC_no_sp = LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                    random_state=0).fit(f_train_og[:,1:],y_train_og)
new_HSIC_no_sp_accuracy0 = lr_model_new_HSIC_no_sp.score(f_train_og[:,1:], y_train_og)
new_HSIC_no_sp_accuracy1 = lr_model_new_HSIC_no_sp.score(f_test_og[:,1:], y_test_og)
new_HSIC_no_sp_accuracy2 = lr_model_new_HSIC_no_sp.score(f_test_t[:,1:], y_test_og)

# put all the results in a dictionary
results_log = {}

results_log['Baseline indist accuracy - spurious corr: '] = baseline_accuracy1
results_log['HSIC Approach indist accuracy - spurious corr: '] = new_HSIC_sp_accuracy1

results_log['Baseline ood accuracy - spurious corr: '] = baseline_accuracy2 
results_log['HSIC Approach ood accuracy- spurious corr: '] = new_HSIC_sp_accuracy2    

results_log['Baseline indist accuracy - no spurious corr: '] = baseline_og_accuracy1
results_log['HSIC Approach indist accuracy - no spurious corr: '] = new_HSIC_no_sp_accuracy1

results_log['Baseline ood accuracy - no spurious corr: '] = baseline_transf_accuracy2            
results_log['HSIC Approach ood accuracy - no spurious corr: '] = new_HSIC_no_sp_accuracy2   


results_log
 


/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

CPU times: user 13min 59s, sys: 34min 25s, total: 48min 24s
Wall time: 46.6 s


{'Baseline indist accuracy - spurious corr: ': 0.8477,
 'HSIC Approach indist accuracy - spurious corr: ': 0.8221,
 'Baseline ood accuracy - spurious corr: ': 0.1579,
 'HSIC Approach ood accuracy- spurious corr: ': 0.301,
 'Baseline indist accuracy - no spurious corr: ': 0.8287,
 'HSIC Approach indist accuracy - no spurious corr: ': 0.8115,
 'Baseline ood accuracy - no spurious corr: ': 0.6206,
 'HSIC Approach ood accuracy - no spurious corr: ': 0.4599}

<br>

<br>

# Compare with PISCO Results on CIFAR10 

In [6]:
%%time

import numpy as np
from sklearn.linear_model import LogisticRegression
from numpy import linalg as LA
import torch
from numpy import load
import sys, json
from itertools import product
from sklearn import preprocessing


# Function for binarizing labels
def binarize(y):    
    y = np.copy(y) > 5
    return y.astype(int)

# Function for creating spurious correlations
def create_spurious_corr(z, z_t, y_og, spu_corr= 0.1, binarize_label=True):
    y_bin = binarize(y_og)
    mod_labels = np.logical_xor(y_bin, np.random.binomial(1, spu_corr, size=len(y_bin)))
    
    modified_images = z_t[mod_labels]
    unmodified_images = z[~mod_labels]
    all_z = np.concatenate((modified_images, unmodified_images), axis=0)
    
    all_img_labels = None
    
    if binarize_label:
        modified_imgs_labels = y_bin[mod_labels]
        unmodified_imgs_labels = y_bin[~mod_labels]
        all_img_labels = np.concatenate((modified_imgs_labels, unmodified_imgs_labels), axis=None)
    else:
        modified_imgs_labels = y_og[mod_labels]
        unmodified_imgs_labels = y_og[~mod_labels]
        all_img_labels = np.concatenate((modified_imgs_labels, unmodified_imgs_labels), axis=None)    
        
    return all_z, all_img_labels 
    


# call this function to get experiments results for different parameters
def get_exp_results(alpha = 1.0, seed=0, lamda=1, extractor='resnet', transf_type='rotated', 
                    dataset='cifar10', eta=0.95):
    
    np.random.seed(seed)
    
    # Load saved image features
    z_train_og = load('./data/Z_train_og_cifar10_simclr.npy')
    z_train_t = load('./data/Z_train_rotated_cifar10_simclr.npy')

    z_test_og = load('./data/Z_test_og_cifar10_simclr.npy')
    z_test_t = load('./data/Z_test_rotated_cifar10_simclr.npy')

    y_train_og = load('./data/train_labels_cifar10.npy')

    y_test_og = load('./data/test_labels_cifar10.npy')
    
    
    # Create spurious correlations on train and test sets
    z_train, train_labels = create_spurious_corr(z_train_og, z_train_t, y_train_og, 
                                             spu_corr= alpha, binarize_label=False)

    z_test_indist, indist_test_labels = create_spurious_corr(z_test_og, z_test_t, y_test_og, 
                                                             spu_corr= alpha, binarize_label=False)

    z_test_ood, ood_test_labels = create_spurious_corr(z_test_og, z_test_t, y_test_og, 
                                                             spu_corr= 1-alpha, binarize_label=False)
  

    # concatenate original and colored features
    z_train_og_t = np.concatenate((z_train_og, z_train_t), axis=0)
    t_train_labels = np.concatenate((np.zeros(len(z_train_og)), np.ones(len(z_train_t))), axis=None) 
    z_test_og_t = np.concatenate((z_test_og, z_test_t), axis=0)
    t_test_labels = np.concatenate((np.zeros(len(z_test_og)), np.ones(len(z_test_t))), axis=None) 
    

    # Obtain prediction coefficients of color
    lr_model_t = LogisticRegression(random_state=0).fit(z_train_og_t, t_train_labels)
    t_coefficients = lr_model_t.coef_.reshape(-1,1)
    theta_1 = t_coefficients / np.linalg.norm(t_coefficients)

    # Prediction Accuracies on image features extracted using a baseline model (mlp)
    logistic_regression_on_baseline = LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                                  random_state=0).fit(z_train,train_labels)                                                                                     
    baseline_accuracy0 = logistic_regression_on_baseline.score(z_train, train_labels)
    baseline_accuracy1 = logistic_regression_on_baseline.score(z_test_indist, indist_test_labels)
    baseline_accuracy2 = logistic_regression_on_baseline.score(z_test_ood, ood_test_labels)   
    
    # Trained on original baseline features, tested on colored features - no spurious correlations here
    logistic_regression_on_baseline_og = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                                                     random_state=0).fit(z_train_og,y_train_og)                                                                                     
    baseline_og_accuracy0 = logistic_regression_on_baseline_og.score(z_train_og, y_train_og)
    baseline_og_accuracy1 = logistic_regression_on_baseline_og.score(z_test_og, y_test_og)
    baseline_transf_accuracy2 = logistic_regression_on_baseline_og.score(z_test_t, y_test_og)

    # Find P, get post-processed features, and perform predictions
    k = int(z_train_og_t.shape[1]*eta) # % of original number of features
    n = z_train_og_t.shape[0]

    delta_z_matrix = z_train_og - z_train_t 

    M = - z_train_og_t.T @ z_train_og_t/n + lamda * delta_z_matrix.T @ delta_z_matrix / (n // 2 ) 

    # Performing SVD to get eigenvectors and eigenvalues
    eigenvalues, eigenvectors = LA.eigh(M)

    # Forming P from eigenvectors and coeficients of color
    P_1 = theta_1

    P_2 = eigenvectors[:,:(k-1)]

    P = np.concatenate((P_1,P_2), axis=1)
    

    # Obtain post-processed features
    f_train_og = z_train_og @ P  
    f_train = z_train @ P
    f_test_indist = z_test_indist @ P
    f_test_ood = z_test_ood @ P
    f_test_og = z_test_og @ P
    f_test_t = z_test_t @ P
    f_test_og_t = z_test_og_t @ P
    
    

    # Correlation Matrix Analysis
    # concatenate transformation labels with f_test_og_t
    t_labels_f_test_og_t = np.concatenate((t_test_labels.reshape(-1,1), f_test_og_t), axis=1)
    t_labels_z_test_og_t = np.concatenate((t_test_labels.reshape(-1,1), z_test_og_t), axis=1)
    corr_matrix = np.corrcoef(t_labels_f_test_og_t.T)
    corr_z_matrix = np.corrcoef(t_labels_z_test_og_t.T)
    corr_special = np.abs(corr_matrix[0,1])
    corr_ns_f_norm = np.sqrt((corr_matrix[0,5:]**2).mean()) 
    z_corr_ns_f_norm = np.sqrt((corr_z_matrix[0,:]**2).mean()) 
     
     
    

    # Classification task using all post-processed features except style features   
    lr_model_pisco_sp = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                                        random_state=0).fit(f_train[:,1:],train_labels)
    pisco_sp_accuracy0 = lr_model_pisco_sp.score(f_train[:,1:], train_labels)
    pisco_sp_accuracy1 = lr_model_pisco_sp.score(f_test_indist[:,1:], indist_test_labels)
    pisco_sp_accuracy2 = lr_model_pisco_sp.score(f_test_ood[:,1:], ood_test_labels)
    
    # trained on original post-processed features, tested on transformed post-processed 
    # features without style features   
    lr_model_pisco_no_sp = LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                        random_state=0).fit(f_train_og[:,1:],y_train_og)
    pisco_no_sp_accuracy0 = lr_model_pisco_no_sp.score(f_train_og[:,1:], y_train_og)
    pisco_no_sp_accuracy1 = lr_model_pisco_no_sp.score(f_test_og[:,1:], y_test_og)
    pisco_no_sp_accuracy2 = lr_model_pisco_no_sp.score(f_test_t[:,1:], y_test_og)
    
    # put all the results in a dictionary
    results_log = {}
    results_log['Baseline indist accuracy - spurious corr: '] = baseline_accuracy1
    results_log['PISCO indist accuracy - spurious corr: '] = pisco_sp_accuracy1

    results_log['Baseline ood accuracy - spurious corr: '] = baseline_accuracy2 
    results_log['PISCO ood accuracy- spurious corr: '] = pisco_sp_accuracy2    

    results_log['Baseline indist accuracy - no spurious corr: '] = baseline_og_accuracy1
    results_log['PISCO indist accuracy - no spurious corr: '] = pisco_no_sp_accuracy1

    results_log['Baseline ood accuracy - no spurious corr: '] = baseline_transf_accuracy2            
    results_log['PISCO ood accuracy - no spurious corr: '] = pisco_no_sp_accuracy2   

    
    return results_log



# if __name__ == "__main__":
#     ITERS = range(10)
#     datasets = ['mnist']
#     extractors= ['mlp']
#     transf_types = ['colored']
#     alphas = [0.5, 0.75, 0.90, 0.95, 0.99,1.0]
#     lamdas= [0,1,10,50] 
#     etas = [0.90]

#     grid = list(product(datasets, extractors, transf_types, alphas, lamdas,etas,ITERS))
    
#     i = int(float(sys.argv[1]))
#     dataset, extractor, transf_type, alpha, lamda, eta, ITER = grid[i]
    

#     results_log = get_exp_results(alpha = alpha, seed=ITER, lamda=lamda, extractor=extractor, 
#                                   transf_type=transf_type, dataset=dataset, eta=eta)

#     with open(f'summary_mnist/summary_{i}.json', 'w') as fp:
#         json.dump(results_log, fp)



get_exp_results(alpha = 1.0, seed=0, lamda=1, extractor='simclr', transf_type='rotated', dataset='cifar10', eta=0.95) 
        

/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/software/anaconda3/envs/invariance_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

CPU times: user 16min 48s, sys: 40min 21s, total: 57min 9s
Wall time: 56 s
Parser   : 115 ms


{'Baseline indist accuracy - spurious corr: ': 0.8477,
 'PISCO indist accuracy - spurious corr: ': 0.8029,
 'Baseline ood accuracy - spurious corr: ': 0.1579,
 'PISCO ood accuracy- spurious corr: ': 0.6714,
 'Baseline indist accuracy - no spurious corr: ': 0.8287,
 'PISCO indist accuracy - no spurious corr: ': 0.8209,
 'Baseline ood accuracy - no spurious corr: ': 0.6206,
 'PISCO ood accuracy - no spurious corr: ': 0.6733}